## Extracción de datos música Punk

In [1]:
#Importar
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
#Acceso: client id, client secret
Client_ID = "bd39a5f17be2422090f8beab3ab83461"
Client_secret = "dacb9e8f9b1c4ad9be7dd105496bc81f"

"redirect URI = http://127.0.0.1:8000/callback"

'redirect URI = http://127.0.0.1:8000/callback'

In [3]:
# Autenticacion
auth_manager = SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
# Función para etraer todos los resultados
def extraer_datos_por_anios(sp, genres, anios, max_por_anio=1000):
    results = []
    artistas = set()  

    for anio in anios:
        print(f"\n Extrayendo datos del año {anio}")

        for genre in genres:
            print(f" Género: {genre}")

            offset = 0
            obtenidos = 0

            while obtenidos < max_por_anio:
                response = sp.search(
                    q=f'genre:{genre} year:{anio}',
                    type='track,album',
                    limit=50,
                    offset=offset
                )

                tracks = response['tracks']['items']
                albums = response['albums']['items']

                if not tracks and not albums:
                    break  

                # Canciones
                for item in tracks:
                    if obtenidos >= max_por_anio:
                        break

                    artista = item['artists'][0]['name']
                    artistas.add(artista)

                    results.append({
                        'name_track': item['name'],
                        'name_artist': artista,
                        'genre': genre,
                        'type': 'track',
                        'year': item['album']['release_date'][:4]
                    })
                    obtenidos += 1

                # Álbumes
                for item in albums:
                    if obtenidos >= max_por_anio:
                        break

                    artista = item['artists'][0]['name']
                    artistas.add(artista)

                    results.append({
                        'name_track': item['name'],
                        'name_artist': artista,
                        'genre': genre,
                        'type': 'album',
                        'year': item['release_date'][:4]
                    })
                    obtenidos += 1

                offset += 50  

            print(f" Total extraído {anio} ({genre}): {obtenidos}")

    return results, list(artistas)

In [7]:
genres = ['punk', 'ska punk']
anios = [2010, 2015, 2020, 2025]

results, artistas = extraer_datos_por_anios(
    sp=sp,
    genres=genres,
    anios=anios,
    max_por_anio=1000
)

print(len(results))
print(len(artistas))


 Extrayendo datos del año 2010
 Género: punk
 Total extraído 2010 (punk): 1000
 Género: ska punk
 Total extraído 2010 (ska punk): 2

 Extrayendo datos del año 2015
 Género: punk
 Total extraído 2015 (punk): 1000
 Género: ska punk
 Total extraído 2015 (ska punk): 6

 Extrayendo datos del año 2020
 Género: punk
 Total extraído 2020 (punk): 1000
 Género: ska punk
 Total extraído 2020 (ska punk): 9

 Extrayendo datos del año 2025
 Género: punk
 Total extraído 2025 (punk): 1000
 Género: ska punk
 Total extraído 2025 (ska punk): 0
4017
1424


In [6]:
# Extracción datos last.fm
api_key = "a03eaf07fb8aba070de6cbf312f88b25"
base_url = "http://ws.audioscrobbler.com/2.0/"

In [8]:
# Función extraer datos
import requests
import time

def obtener_info_artistas_lastfm(artistas, api_key, delay=0.25):

    base_url = "http://ws.audioscrobbler.com/2.0/"
    resultados = []

    for artista in artistas:
        params = {
            "method": "artist.getInfo",
            "artist": artista,
            "api_key": api_key,
            "format": "json"
        }

        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "artist" not in data:
                print(f"  {artista} No encontrado.")
                continue

            artist_data = data["artist"]

            resultados.append({
                "artista": artista,
                "biografia": artist_data.get("bio", {}).get("summary", ""),
                "Oyentes": int(artist_data.get("stats", {}).get("listeners", 0)),
                "N_Reproducciones": int(artist_data.get("stats", {}).get("playcount", 0)),
                "artistas_similares": [
                    a["name"] for a in artist_data.get("similar", {}).get("artist", [])
                ]
            })

            time.sleep(delay) 

        except Exception as e:
            print(f"Error con {artista}: {e}")

    return resultados

In [9]:
info_artistas = obtener_info_artistas_lastfm(artistas, api_key)

print(len(info_artistas))
info_artistas[0]

   I Promised The World No encontrado.
  CNQR+ No encontrado.
1422


{'artista': 'Ghost Runner On Third',
 'biografia': 'A veteran of the local pop-punk scene, Tacoma’s own Ghost Runner On Third has been at it since 2002.\n\nAfter releasing its EP “Speak Your Dreams” in 2005, the quintet toured the West Coast. The band’s latest incarnation features Adam Fuller (vocals and keys) and Preston Kristof (bass), in addition to three original members, Colin Willard (guitar and backup vocals), Mike Johnson (guitar) and Ryan Polson (drums).\n\nThey\'ve also had former members such as: <a href="https://www.last.fm/music/Ghost+Runner+On+Third">Read more on Last.fm</a>',
 'Oyentes': 23224,
 'N_Reproducciones': 336544,
 'artistas_similares': ["We're Not Friends Anymore",
  'GunsLikeGirls',
  'Her Words Kill',
  'Dear Whoever',
  'Unwanted Superheroes']}

In [ ]:
# Variables con datos obtenidos:
results
artistas
info_artistas

In [11]:
import pandas as pd

In [12]:
df_results = pd.DataFrame(results)

In [15]:
df_results

,name_track,name_artist,genre,type,year
0,Perfect Weapon,Black Veil Brides,punk,track,2010
1,Whore,Get Scared,punk,track,2010
2,"With Ears to See, and Eyes to Hear",Sleeping With Sirens,punk,track,2010
3,Whiskey and Gin,The Killigans,punk,track,2010
4,A New Today,Endymion,punk,track,2010
...,...,...,...,...,...
4012,I like the way u talk to me,NESYA,punk,track,2025
4013,angelonia,grcz,punk,track,2025
4014,Softcrush,CNQR+,punk,track,2025
4015,Fine,Ductape,punk,track,2025


In [21]:
df_results.to_csv("spotify_base_datos_final_punk.csv", index=False)

In [13]:
df_artistas = pd.DataFrame(artistas)

In [16]:
df_artistas

,0
0,Ghost Runner On Third
1,The Meteors
2,He Said She's Dead
3,Kili
4,The Genesis Projection
...,...
1419,Down To Nothing
1420,Covet
1421,Feed Me Jack
1422,SR-71


In [20]:
df_artistas.to_csv("lastfm_unique_artist_punk.csv", index=False)

In [14]:
df_info_artistas = pd.DataFrame(info_artistas)

In [19]:
df_info_artistas.to_csv("lastfm_unique_artist_punk.csv", index=False)

In [17]:
df_info_artistas

,artista,biografia,Oyentes,N_Reproducciones,artistas_similares
0,Ghost Runner On Third,"A veteran of the local pop-punk scene, Tacoma’...",23224,336544,"[We're Not Friends Anymore, GunsLikeGirls, Her..."
1,The Meteors,There are at least 3 bands called The Meteors....,179527,3932362,"[Demented Are Go!, Batmobile, Nekromantix, Mad..."
2,He Said She's Dead,"He Said, She's Dead was born out of high schoo...",5285,69889,"[Forever Wings Fold, Reclaim The Fallen, Oh Sw..."
3,Kili,"<a href=""https://www.last.fm/music/Kili"">Read...",29441,498244,"[Artie J, kwn, Pino, isaiah kaleo, kili, gabbinz]"
4,The Genesis Projection,"Sergio, after having passed few years playing ...",3941,34476,"[Tomcat & Rudeboy, Raw Saints, Divato, Negativ..."
...,...,...,...,...,...
1417,Down To Nothing,Down to Nothing (or DTN) is a straight edge ha...,83303,4171289,"[Guns Up!, Backtrack, Bracewar, Cruel Hand, Ro..."
1418,Covet,There are two bands with the name covet:\n\nCo...,195974,4113182,"[Yvette Young, CHON, Floral, Clever Girl, Via ..."
1419,Feed Me Jack,Feed Me Jack were an Indie/Progressive Rock gr...,132205,2855285,"[Bobbing, Hikes, TTNG, Delta Sleep, Floral]"
1420,SR-71,"Formed in Baltimore, MD, United States in 1998...",449980,6562938,"[Sugarcult, American Hi-Fi, Bowling for Soup, ..."
